## Aspire + AlphaCSC pipeline

In this notebook we will run automated spike detection pipeline and visualize the results. We will use simulation described in `0_simulation.ipynb`.

NOTE: To import the megspikes modules we should change the working directory if we are running this example from the cloned GitHub repository.

In [ ]:
import os
from pathlib import Path

import matplotlib.pylab as plt
import mne
import numpy as np

# change to the root directory of the project
if os.getcwd().split("/")[-1] == "examples":
    os.chdir('..')

from megspikes.simulation.simulation import Simulation

# Setup the path for the simulation
sample_path = Path(os.getcwd()) / 'examples' / 'data' / '2_aspire_alphacsc_pipeline'
sample_path.mkdir(exist_ok=True, parents=True)

import logging
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test debug")
logging.info("test info")
%load_ext autoreload
%autoreload 2

## Simulate data

For more information see `0_simulation.ipynb`

In [ ]:
sim = Simulation(sample_path)
sim.simulate_dataset()
sim

## Run Aspire-AlphaCSC Pipeline

In [ ]:
from megspikes.pipeline import aspire_alphacsc_pipeline

from sklearn import set_config
set_config(display='diagram')
set_config(print_changed_only=False)

First, we should specify pipeline parameters. All parameters are listed in the `aspire_alphacsc_default_params.yml`.

In [ ]:
params = {
    'n_ica_components': 5,
    'n_runs': 3,
    'runs': [0, 1, 2],
    'n_atoms': 2,
    'PeakDetection': {'width': 2},
    'CleanDetections': {'n_cleaned_peaks': 50},
    'SelectAlphacscEvents': {
        'z_hat_threshold': 1.,
        'z_hat_threshold_min': 0.1}
}

Using this parameters we create `sklear.Pipeline`

In [ ]:
pipe = aspire_alphacsc_pipeline(sim.case_manager, params)
pipe

We can also save it as an __HTML__ document

In [ ]:
from sklearn.utils import estimator_html_repr
with open('my_estimator.html', 'w') as f:  
    f.write(estimator_html_repr(pipe))

There is not input for this pipline, so we can run it with `X=None`. As an output we aquire `xr.Dataset` with all results and `mne.io.Raw` file.

In [ ]:
dataset, raw = pipe.fit_transform(None)

Now we can also load created dataset.

In [ ]:
import xarray as xr
dataset = xr.open_dataset(sample_path / 'sample' / 'sample_alphacsc_results.nc')

# Visualize detection


In [ ]:
from megspikes.visualization.visualization import DetectionsViewer
pp = DetectionsViewer(dataset, sim.case_manager)

## Plot ICA components


In [ ]:
app = pp.view_ica()
app.show()

## Plot ICA sources and detections


In [ ]:
app = pp.view_ica_sources_and_peaks()
app.show()

## Plot ICA fast RAP-MUSIC localizatons

In [ ]:
dataset

## Plot AlphaCSC atoms

In [ ]:
pp.data.alphacsc_atoms

In [ ]:
app = pp.view_alphacsc_atoms()
app.show()

## Plot AlphaCSC selected evets

In [ ]:
app = pp.view_alphacsc_clusters(sim.raw_simulation.copy())
app.show()

# Localize detected clusters

In [ ]:
from megspikes.pipeline import read_detection_iz_prediction_pipeline
from sklearn import set_config
set_config(display='diagram')
set_config(print_changed_only=False)

In [ ]:
params = {
    'PrepareClustersDataset': {'detection_sfreq': 200.}
}
pipe = read_detection_iz_prediction_pipeline(sim.case_manager, params)
pipe

In [ ]:
clusters, raw = pipe.fit_transform((dataset, sim.raw_simulation.copy()))

In [ ]:
clusters

In [ ]:
import xarray as xr
clusters = xr.open_dataset(sample_path / 'sample' / 'sample_alphacsc_cluster_results.nc')

## View clusters using Cluster Slope Viewer

In [ ]:
import mne
mne.viz.set_3d_backend('pyvista')

In [ ]:
from megspikes.visualization.visualization import ClusterSlopeViewer
import panel as pn


In [ ]:
%matplotlib qt5
pc = ClusterSlopeViewer(clusters, sim.case_manager)
pc.data.clusters

In [ ]:
pc.view()

## Evaluate Irritative Zone prediction

In [ ]:
from megspikes.scoring.scoring import ScoreIZPrediction
scorer = ScoreIZPrediction()

In [ ]:
scorer.score(clusters, sim.mni_resection, 'peak')

In [ ]:
scorer.score(clusters, sim.mni_resection, 'slope')

### Plot resection and prediction

In [ ]:
%matplotlib inline
from nilearn import plotting
fig, ax = plt.subplots(figsize=(15, 7))

display = plotting.plot_glass_brain(
            None, display_mode='lzry', figure=fig, axes=ax)

display.add_markers(scorer.detection_mni, marker_color='tomato', alpha=0.2)
display.add_markers(sim.mni_resection, marker_color='indigo', alpha=0.6)